In [1]:
!pip install -q -U "transformers>=4.45.0,<5.0.0" huggingface_hub datasets peft trl bitsandbytes accelerate wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 40.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.31.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-adk 1.21.0 requires google-cloud-bigquery-storage>=2.0.0, which is not installed.


In [2]:
import wandb
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
# Add these tokens in Kaggle's "Secrets" tab (Add-ons -> Secrets)
login(token=user_secrets.get_secret("HF_TOKEN"))
wandb.login(key=user_secrets.get_secret("WANDB_API_KEY"))

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: [wandb.login()] Using explicit session credentials for https://api.wandb.ai.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shreyashgaur01 (Nyaya-LLM) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
import torch
import os
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer, SFTConfig

os.environ["WANDB_ENTITY"] = "Nyaya-LLM"
os.environ["WANDB_PROJECT"] = "nyaya-llm-phase1"

MODEL_NAME = "Qwen/Qwen3-4B-Instruct-2507"
DATA_PATH = "/kaggle/input/datasets/shreyashgaur221/nyaya-llm-dataset/all_acts_train.jsonl"
VAL_PATH = "/kaggle/input/datasets/shreyashgaur221/nyaya-llm-dataset/all_acts_val.jsonl"
OUTPUT_DIR = "/kaggle/working/results/lora_phase1_qwen3_4b"
MAX_SEQ_LENGTH = 1024


def format_prompt(example):
    return f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['output']}"


def main():
    print("Loading dataset...")
    dataset = load_dataset("json", data_files={"train": DATA_PATH, "validation": VAL_PATH})

    print("Loading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token

    print("Loading base model in fp16...")
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
    )

    model.gradient_checkpointing_enable()
    model.enable_input_require_grads()
    model.config.use_cache = False

    print("Applying LoRA adapters...")
    peft_config = LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
    )
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()

    print("Setting up Trainer...")
    training_args = SFTConfig(
        output_dir=OUTPUT_DIR,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=8,
        eval_strategy="steps",
        eval_steps=100,
        logging_steps=25,
        num_train_epochs=2,
        learning_rate=2e-4,
        fp16=True,
        bf16=False,
        max_grad_norm=1.0,       # HF default — Phi used this implicitly
        warmup_ratio=0.03,       # prevents gradient spike at step 1
        optim="adamw_torch",
        save_strategy="epoch",
        save_total_limit=4,
        max_length=MAX_SEQ_LENGTH,
        report_to="wandb",
        run_name="lora-qwen3-4b",
        dataloader_pin_memory=False,
    )

    trainer = SFTTrainer(
        model=model,
        processing_class=tokenizer,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"],
        formatting_func=format_prompt,
        args=training_args,
    )

    print("Starting training...")
    trainer.train()

    print("Saving model adapter...")
    trainer.model.save_pretrained(OUTPUT_DIR)
    tokenizer.save_pretrained(OUTPUT_DIR)
    print("Qwen3-4B LoRA Training Complete.")


if __name__ == "__main__":
    main()

2026-02-19 09:18:57.464209: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771492737.673097      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771492737.739296      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771492738.200178      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771492738.200236      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771492738.200239      24 computation_placer.cc:177] computation placer alr

Loading dataset...


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Loading tokenizer...


...

The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151645}.


Starting training...


wandb: WARNING Changes to your `wandb` environment variables will be ignored because your `wandb` session has already started. For more information on how to modify your settings with `wandb.init()` arguments, please refer to https://wandb.me/wandb-init.
wandb: Tracking run with wandb version 0.25.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20260219_092010-7tc6v6qa
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run lora-qwen3-4b
wandb: ⭐️ View project at https://wandb.ai/Nyaya-LLM/nyaya-llm-phase1
wandb: 🚀 View run at https://wandb.ai/Nyaya-LLM/nyaya-llm-phase1/runs/7tc6v6qa


Step,Training Loss,Validation Loss
100,1.307000,1.179864
200,1.238900,1.128065
300,1.188200,1.096982
400,1.224400,1.062492
500,1.155300,1.031854
600,1.109200,1.001664
700,1.087800,0.977659
800,1.071700,0.952483
900,1.038700,0.928753
1000,0.923200,0.908095


Saving model adapter...
Qwen3-4B LoRA Training Complete.
